# Montecarlo assessment

A Montecarlo simulation is a way to assess uncertainties and robustness given probabilities on the model inputs. 

Here we illustrate how it works with a design Montecarlo. It is also called robust design because we can  evaluate the impact on the designed characteristics (here the `exchanger.surface`).

In [ ]:
from cosapp.drivers import MonteCarlo, RungeKutta, NonLinearSolver
from cosapp.recorders import DataFrameRecorder
from cosapp.utils.distributions import Normal

from cpu.systems import CPUSystem

In [ ]:
cpu = CPUSystem("cpu")

In [ ]:
mc = cpu.add_driver(MonteCarlo('mc'))

design = mc.add_child(NonLinearSolver('design'))
design.extend(cpu.design_methods["exchanger_surface"])
design.runner.set_values({"T_cpu": 80., "cpu.usage": 100.})

mc.add_recorder(DataFrameRecorder(includes=['*']))
mc.draws = 100

# parameters for uncertainties in the data
T_air = cpu.fan.inwards.get_details('T_air')
# Set the distribution around the current value
T_air.distribution = Normal(best=1, worst=-0.5)
mc.add_random_variable('fan.T_air')
mc.add_response(['fan.tension', 'cpu.usage', 'T_cpu'])

In [ ]:
cpu.run_drivers()
mc.recorder.export_data()